In [416]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tensorflow import keras
from PIL import Image
import scipy.io
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD


In [417]:
def image():
    fichier = open('../../asset/Train/Keras/test/wider_face_train_bbx_gt.txt ', 'r')
    mat = scipy.io.loadmat('../../asset/Train/Keras/test/wider_face_train.mat')

    numCat = random.randint(0,len(mat['file_list'])-1)
    numObject =  random.randint(0,len(mat['file_list'][numCat][0])-1)
    numCatReel = mat['file_list'][numCat][0][numObject][0][0].split("_")[0]
    for dir in os.listdir('../../asset/Train/Keras/WIDER_train/images'):
        if dir.split("-")[0] == numCatReel:
            break
    chemin=dir+"/"+mat['file_list'][numCat][0][numObject][0][0]+".jpg"
    return chemin,mat['face_bbx_list'][numCat][0][numObject][0]


In [429]:
def objectTrain(nbImages):
    lImg = []
    lLabel = []
    for i in range(nbImages):
        chemin,box = image()
        img = tf.keras.preprocessing.image.load_img("../../asset/Train/Keras/WIDER_train/images/"+chemin)
        img = keras.preprocessing.image.img_to_array(img)
        print(img.shape)
        lImg.append(img)
        lLabel.append(box)
    lImg = np.array(lImg)
    lLabel = np.array(lLabel)
    return lImg, lLabel 

In [431]:
def modele():
        cnn = keras.models.Sequential()

        cnn.add(keras.layers.Convolution2D(32, 3, padding='same',  activation='relu', input_shape=(1700,1024,1)))
        cnn.add(keras.layers.Convolution2D(32, 3, 3, activation='relu'))
        cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        cnn.add(keras.layers.Dropout(0.25))

        cnn.add(keras.layers.Convolution2D(64, 3, padding='same',  activation='relu'))
        cnn.add(keras.layers.Convolution2D(64, 3, 3, activation='relu'))
        cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        cnn.add(keras.layers.Dropout(0.25))

        cnn.add(keras.layers.Flatten())
        cnn.add(keras.layers.Dense(512 ,  activation='relu'))
        cnn.add(keras.layers.Dropout(0.5))
        cnn.add(keras.layers.Dense(15, activation = 'softmax'))
        #cnn.summary()

        opt = SGD(lr=0.01, decay=1e-65, momentum=0.92, nesterov=True)

        cnn.compile(loss='categorical_crossentropy',
                         optimizer=opt,
                         metrics='accuracy')
        return cnn

In [430]:
objects, labels = objectTrain(100)

(465, 1024, 3)
(681, 1024, 3)
(684, 1024, 3)
(585, 1024, 3)
(683, 1024, 3)
(1024, 1024, 3)
(1408, 1024, 3)
(682, 1024, 3)
(576, 1024, 3)
(751, 1024, 3)
(898, 1024, 3)
(680, 1024, 3)
(1535, 1024, 3)
(683, 1024, 3)
(576, 1024, 3)
(768, 1024, 3)
(1434, 1024, 3)
(682, 1024, 3)
(1460, 1024, 3)
(683, 1024, 3)
(1275, 1024, 3)
(656, 1024, 3)
(722, 1024, 3)
(640, 1024, 3)
(679, 1024, 3)
(1517, 1024, 3)
(567, 1024, 3)
(683, 1024, 3)
(676, 1024, 3)
(818, 1024, 3)
(820, 1024, 3)
(698, 1024, 3)
(821, 1024, 3)
(717, 1024, 3)
(1504, 1024, 3)
(1540, 1024, 3)
(683, 1024, 3)
(716, 1024, 3)
(577, 1024, 3)
(1069, 1024, 3)
(768, 1024, 3)
(1366, 1024, 3)
(912, 1024, 3)
(683, 1024, 3)
(1356, 1024, 3)
(1354, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(684, 1024, 3)
(685, 1024, 3)
(671, 1024, 3)
(640, 1024, 3)
(683, 1024, 3)
(497, 1024, 3)
(1416, 1024, 3)
(768, 1024, 3)
(1169, 1024, 3)
(1026, 1024, 3)
(695, 1024, 3)
(680, 1024, 3)
(618, 1024, 3)
(911, 1024, 3)
(601, 1024, 3)
(577, 1024, 3)
(670, 1024, 3)
(682, 102

In [432]:
def train(n_epochs=20, batch=32):
    
    cnn = modele()
    cnn.fit(objects, labels,
        batch_size=batch,
        epochs=n_epochs, shuffle=True)


In [433]:
train()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).